### Environment Setup

Responsible for adjusting `sys.path`, importing essential libraries and configuring the environment for use with Webots.

In [1]:
import sys
import os
sys.path = [p for p in sys.path if 'controller' not in p]

# Insert the correct Webots controller at the beginning
sys.path.insert(0, 'C:\\Program Files\\Webots\\lib\\controller\\python')
sys.path.insert(0, '/Applications/Webots.app/Contents/lib/controller/python') #Mila


import math
import matplotlib.pyplot as plt
import random as pyrandom
from controller import Supervisor
import random
import numpy as np
from controller import Supervisor, Lidar
from controller import Robot

### Velocity Control

This script handles the speed control of the robot, using `cmd_vel`. This file can serve as a motion control module.

In [2]:
try:
    from controllers.utils import cmd_vel
except ImportError:
    print("Warning: controllers.utils.cmd_vel not found. Using dummy function.")
    def cmd_vel(supervisor, lv, av):
        # Dummy implementation for testing without the utility file
        left_motor = supervisor.getDevice("left wheel motor")
        right_motor = supervisor.getDevice("right wheel motor")
        wheel_radius = 0.02 # Assuming a standard wheel radius
        axle_track = 0.0565 # Assuming a standard axle track
        left_velocity = (lv - av * axle_track / 2.0) / wheel_radius
        right_velocity = (lv + av * axle_track / 2.0) / wheel_radius
        left_motor.setVelocity(left_velocity)
        right_motor.setVelocity(right_velocity)

### Global Configuration

This file defines all global constants used in the project, including robot specs, tunnel parameters, wall dimensions, and map limits. Centralizing these values makes the code easier to adjust and maintain.


In [3]:

# --- Configuration ---
ROBOT_NAME = "e-puck"
ROBOT_RADIUS = 0.035  # meters

# Tunnel clearance relative to robot diameter (2 * ROBOT_RADIUS)
MIN_CLEARANCE_FACTOR = 2.5
MAX_CLEARANCE_FACTOR = 4.0

# Minimum required clearance for the robot to pass an obstacle
MIN_ROBOT_CLEARANCE_FACTOR = 2.1
MIN_ROBOT_CLEARANCE = ROBOT_RADIUS * MIN_ROBOT_CLEARANCE_FACTOR

# Minimum distance between obstacles
MIN_OBSTACLE_DISTANCE_FACTOR = 4.0
MIN_OBSTACLE_DISTANCE = ROBOT_RADIUS * MIN_OBSTACLE_DISTANCE_FACTOR

# Number of obstacles to place in the tunnel
NUM_OBSTACLES = 2


BASE_SPEED = 0.1
BASE_WALL_LENGTH = 1.0
WALL_THICKNESS = 0.01
WALL_HEIGHT = 0.07
MIN_CURVE_ANGLE = math.radians(10)
MAX_CURVE_ANGLE = math.radians(90)
CURVE_SUBDIVISIONS = 30 # Increased subdivisions for smoother curves
TIMEOUT_DURATION = 45.0
NUM_CURVES = 3

# Map boundaries (example values, adjust to your world limits)
MAP_X_MIN, MAP_X_MAX = -2.0, 2.0
MAP_Y_MIN, MAP_Y_MAX = -2.0, 2.0

# Small overlap factor to ensure no gaps in curves
OVERLAP_FACTOR = 1.01 # Adjusted overlap factor based on user feedback

### TunnelBuilder Class – Initialization

This class manages the construction of a configurable tunnel in Webots. It keeps track of all wall segments and obstacles, and interacts directly with the simulation environment using a Supervisor.

In [4]:
class TunnelBuilder:
    def __init__(self, supervisor):
        self.supervisor = supervisor
        self.root_children = supervisor.getRoot().getField("children")
        self.base_wall_distance = 0
        self.walls = []
        self.segments = []
        self.right_walls = []
        self.left_walls = []
        self.obstacles = []

### Wall Creation Method

Creates a wall or obstacle with given position, rotation, and size. The wall is added to the Webots simulation and categorized based on type (`left`, `right`, or `obstacle`).

In [5]:
class TunnelBuilder(TunnelBuilder):
    def create_wall(self, pos, rot, size, wall_type=None):
            wall = f"""Solid {{
                translation {pos[0]} {pos[1]} {pos[2]}
                rotation {rot[0]} {rot[1]} {rot[2]} {rot[3]}
                children [
                    Shape {{
                        appearance Appearance {{
                            material Material {{
                                diffuseColor 1 0 0
                            }}
                        }}
                        geometry Box {{
                            size {size[0]} {size[1]} {size[2]}
                        }}
                    }}
                ]
            }}"""
            self.root_children.importMFNodeFromString(-1, wall)
            if wall_type == 'left':
                self.left_walls.append((pos, rot, size))
            elif wall_type == 'right':
                self.right_walls.append((pos, rot, size))
            elif wall_type == 'obstacle':
                self.obstacles.append((pos, rot, size))

### Tunnel Construction Method

Generates the full tunnel using straight segments and curves. It calculates the transformation matrices for each segment and places them while ensuring they stay within map boundaries.

In [6]:
class TunnelBuilder(TunnelBuilder):
    def build_tunnel(self, num_curves, angle_range=None, clearance=None):
            if clearance is not None:
                tunnel_clearance_factor = clearance
            else:
                tunnel_clearance_factor = pyrandom.uniform(MIN_CLEARANCE_FACTOR, MAX_CLEARANCE_FACTOR)
        
            self.base_wall_distance = ROBOT_RADIUS * tunnel_clearance_factor
            print(f"Building tunnel with clearance factor: {tunnel_clearance_factor:.2f}")
        
            angle_min, angle_max = angle_range if angle_range else (MIN_CURVE_ANGLE, MAX_CURVE_ANGLE)
    
            num_curves = min(num_curves, NUM_CURVES)
            length = BASE_WALL_LENGTH * (1 + pyrandom.uniform(-0.15, 0.15))
            segment_length = length / (num_curves + 1)
    
            T = np.eye(4)
            start_pos = T[:3, 3].copy()
            walls = []
            segments_data = []
    
            if not self._within_bounds(T, segment_length):
                return None, None, 0
            segment_start_pos = T[:3, 3].copy()
            self._add_straight(T, segment_length, walls)
            segment_end_pos = T[:3, 3].copy()
            segment_heading = math.atan2(T[1, 0], T[0, 0])
            segments_data.append((segment_start_pos, segment_end_pos, segment_heading, segment_length))
    
            for _ in range(num_curves):
                angle = pyrandom.uniform(angle_min, angle_max) * pyrandom.choice([1, -1])
    
                if not self._within_bounds_after_curve(T, angle, segment_length):
                    break
                prev_end_pos = T[:3, 3].copy()
                self._add_curve(T, angle, segment_length, walls, segments_data)
                new_start = segments_data[-CURVE_SUBDIVISIONS][0]  # início da curva recém-adicionada
                gap = np.linalg.norm(prev_end_pos - new_start)
                if gap > 1e-3:
                    print(f"[WARNING] Discontinuity between curve segments: gap = {gap:.3f}")
                if not self._within_bounds(T, segment_length):
                    break
                segment_start_pos = T[:3, 3].copy()
                self._add_straight(T, segment_length, walls)
                segment_end_pos = T[:3, 3].copy()
                segment_heading = math.atan2(T[1, 0], T[0, 0])
                segments_data.append((segment_start_pos, segment_end_pos, segment_heading, segment_length))
    
            end_pos = T[:3, 3].copy()
            self._add_obstacles(segments_data, walls)
            self.walls = walls
            self.segments = segments_data
            return start_pos, end_pos, len(walls)

### Distance from Robot to Tunnel Centerline

Helper function to compute the minimum distance from the robot to any segment of the tunnel, used to check alignment with the centerline.


In [7]:
class TunnelBuilder(TunnelBuilder):
    def point_to_segment_distance(self, A, B, P):
        AP = P - A
        AB = B - A
        t = np.clip(np.dot(AP, AB) / np.dot(AB, AB), 0.0, 1.0)
        closest_point = A + t * AB
        return np.linalg.norm(P - closest_point)

### Checking Robot Alignment to Centerline

Checks whether the robot is close enough to the central path of the tunnel, considering both straight and curved segments.


In [8]:
class TunnelBuilder(TunnelBuilder):
    def is_robot_near_centerline(self, robot_position):
            """
            Checks if the robot is close enough to the center line of the tunnel,
            considering all segments (straight + curves).
            """
            robot_xy = np.array(robot_position[:2])
            threshold = self.base_wall_distance + ROBOT_RADIUS
        
            for start, end, _, _ in self.segments:
                dist = self.point_to_segment_distance(start[:2], end[:2], robot_xy)
                if dist <= threshold:
                    return True
            return False #vir aqui 
    
    
        
            robot_xy = np.array(robot_position[:2])
            right_dir = np.array([math.cos(heading - math.pi/2), math.sin(heading - math.pi/2)])
            left_dir = np.array([math.cos(heading + math.pi/2), math.sin(heading + math.pi/2)])
        
            def check_walls(walls, direction):
                for pos, _, _ in walls:
                    wall_xy = np.array(pos[:2])
                    to_wall = wall_xy - robot_xy
                    dist = np.linalg.norm(to_wall)
                    dot = np.dot(to_wall / (dist + 1e-6), direction)
                    if dot > 0.7:
                        expected = self.base_wall_distance
                        if abs(dist - expected) < 0.05:
                            return True
                return False
        
            def check_obstacles(obstacles):
                for pos, _, size in obstacles:
                    wall_xy = np.array(pos[:2])
                    dist = np.linalg.norm(wall_xy - robot_xy)
                    # Tolerância baseada na largura do obstáculo
                    if dist < size[1] / 2 + ROBOT_RADIUS + 0.02:
                        return True
                return False
        
            near_right = check_walls(self.right_walls, right_dir)
            near_left = check_walls(self.left_walls, left_dir)
            near_obstacle = check_obstacles(self.obstacles)
        
            inside = (near_right and near_left) or near_obstacle
            #print(f"[DEBUG] Inside tunnel: {inside} | Right: {near_right}, Left: {near_left}, Obstacle: {near_obstacle}")
            return inside

### Adding Straight Tunnel Segments

Adds straight walls to the left and right of the centerline, with proper spacing and orientation.

In [9]:
class TunnelBuilder(TunnelBuilder):
    def _add_straight(self, T, length, walls):
            # Calculate the heading of the current segment
            heading = math.atan2(T[1, 0], T[0, 0])
            # Add walls on both sides of the centerline
            for side in [-1, 1]:
                # Calculate the wall's center position:
                # Start at current T's position, move half the length along T's x-axis (forward),
                # then move self.base_wall_distance along T's y-axis (sideways), and half height up.
                pos = T[:3, 3] + T[:3, 0] * (length / 2) + T[:3, 1] * (side * self.base_wall_distance) + np.array([0, 0, WALL_HEIGHT / 2])
                # Rotation is around the z-axis based on the heading
                rot = (0, 0, 1, heading)
                # Size of the wall (length, thickness, height)
                size = (length, WALL_THICKNESS, WALL_HEIGHT)
                wall_type = 'left' if side == -1 else 'right'
                self.create_wall(pos, rot, size, wall_type=wall_type)
                walls.append((pos, rot, size)) # Append wall details to the walls list
            # Update T to the end of the straight segment
            T[:3, 3] += T[:3, 0] * length

### Adding Curved Tunnel Segments

Adds a smooth curve to the tunnel by incrementally rotating and moving the transformation matrix. It subdivides the curve into smaller segments for better precision.


In [10]:
class TunnelBuilder(TunnelBuilder):
    def _add_curve(self, T, angle, segment_length, walls, segments_data):
            step = angle / CURVE_SUBDIVISIONS
            centerline_step_length = segment_length / CURVE_SUBDIVISIONS
            r_centerline = self.base_wall_distance
            r_inner_edge = r_centerline - WALL_THICKNESS / 2.0
            r_outer_edge = r_centerline + WALL_THICKNESS / 2.0
    
            for _ in range(CURVE_SUBDIVISIONS):
                T_start_step = T.copy()
                segment_start = T[:3, 3].copy()
    
                T_mid_step = T_start_step @ self._rotation_z(step / 2)
                T_mid_step[:3, 3] += T_mid_step[:3, 0] * (centerline_step_length / 2)
                heading = math.atan2(T_mid_step[1, 0], T_mid_step[0, 0])
                rot = (0, 0, 1, heading)
    
                for side in [-1, 1]:
                    wall_length = r_inner_edge * abs(step) if side == -1 else r_outer_edge * abs(step)
                    wall_length += OVERLAP_FACTOR * WALL_THICKNESS
                    pos = T_start_step[:3, 3] + T_start_step[:3, 1] * (side * self.base_wall_distance) + np.array([0, 0, WALL_HEIGHT / 2]) + T_start_step[:3, 0] * (wall_length / 2)
                    size = (wall_length, WALL_THICKNESS, WALL_HEIGHT)
                    wall_type = 'left' if side == -1 else 'right'
                    self.create_wall(pos, rot, size, wall_type=wall_type)
                    walls.append((pos, rot, size))
                    
    
                T[:] = T_start_step @ self._rotation_z(step)
                T[:3, 3] += T[:3, 0] * centerline_step_length
    
                segment_end = T[:3, 3].copy()
                segment_heading = math.atan2(T[1, 0], T[0, 0])
                segments_data.append((segment_start, segment_end, segment_heading, centerline_step_length))
                #print(f"[CURVE SEGMENT] start: {segment_start}, end: {segment_end}, heading: {math.degrees(segment_heading):.1f}°")

### Placing Obstacles

Randomly places perpendicular obstacles in the middle of straight segments, maintaining safe distance between them.


In [11]:
class TunnelBuilder(TunnelBuilder):
    def _add_obstacles(self, straight_segments_data, walls):
            """
            Place NUM_OBSTACLES perpendicular walls into the middle straight segments.
            straight_segments_data: list of (start_pos, end_pos, heading, length)
            walls: list to append (pos, rot, size) tuples for cleanup/tracking
            """
            # skip entrance & exit straights
            segments = straight_segments_data[1:-1]
            if not segments:
                print("Not enough segments for obstacles.")
                return
    
            used = set()
            placed_positions = []
    
            # half-width of tunnel from centerline
            tunnel_half = ROBOT_RADIUS * MIN_CLEARANCE_FACTOR
            # obstacle span across tunnel, leaving MIN_ROBOT_CLEARANCE on the other side
            obstacle_length = 2 * tunnel_half - MIN_ROBOT_CLEARANCE - WALL_THICKNESS
    
            for _ in range(NUM_OBSTACLES):
                # pick an unused segment index
                choices = [i for i in range(len(segments)) if i not in used]
                if not choices:
                    break
                idx = pyrandom.choice(choices)
                used.add(idx)
    
                start, end, heading, seg_len = segments[idx]
                # choose a point 20–80% along the straight
                d = pyrandom.uniform(0.2 * seg_len, 0.8 * seg_len)
                dir_vec = np.array([math.cos(heading), math.sin(heading), 0.0])
                pos = np.array(start) + dir_vec * d
    
                # pick inner/outer side
                side = pyrandom.choice([-1, +1])
                perp = np.array([-dir_vec[1], dir_vec[0], 0.0])
                shift = side * (MIN_ROBOT_CLEARANCE / 2 + obstacle_length / 2)
                pos += perp * shift
                pos[2] = WALL_HEIGHT / 2.0
    
                # no extra rotation—box X-axis is already perpendicular to the corridor
                rot = (0.0, 0.0, 1.0, heading)
                # size: X = thickness along path, Y = span across path
                size = (WALL_THICKNESS, obstacle_length, WALL_HEIGHT)
    
                # avoid clustering
                if any(np.linalg.norm(pos[:2] - p) < MIN_OBSTACLE_DISTANCE
                       for p in placed_positions):
                    print("Skipping obstacle—too close to another.")
                    continue
    
                # create and record
                wall_type = 'left' if side == -1 else 'right'
                self.create_wall(pos, rot, size, wall_type='obstacle')
    
    
                walls.append((pos, rot, size))
                placed_positions.append(pos[:2].copy())

### Transformation Helpers

Utility functions to generate translation and rotation matrices used during tunnel construction.


In [12]:
class TunnelBuilder(TunnelBuilder):
    def _translation(self, x, y, z):
            # Helper function to create a translation matrix
            M = np.eye(4)
            M[:3, 3] = [x, y, z]
            return M
    
    def _rotation_z(self, angle):
        # Helper function to create a rotation matrix around the z-axis
        c, s = math.cos(angle), math.sin(angle)
        R = np.eye(4)
        R[0, 0], R[0, 1] = c, -s
        R[1, 0], R[1, 1] = s, c
        return R


### Map Bounds Checkers

Verifies whether the new segment or curve remains within the defined simulation boundaries.


In [13]:
class TunnelBuilder(TunnelBuilder):
    def _within_bounds(self, T, length):
            # Check if the end point of a straight segment is within map bounds
            end = T[:3, 3] + T[:3, 0] * length
            return MAP_X_MIN <= end[0] <= MAP_X_MAX and MAP_Y_MIN <= end[1] <= MAP_Y_MAX
    
    def _within_bounds_after_curve(self, T, angle, seg_len):
        # Check if the end point after a curved segment is within map bounds
        tempT = T.copy()
        step = angle / CURVE_SUBDIVISIONS
        centerline_step_length = seg_len / CURVE_SUBDIVISIONS
        for _ in range(CURVE_SUBDIVISIONS):
            tempT = tempT @ self._rotation_z(step)
            tempT[:3, 3] += tempT[:3, 0] * centerline_step_length
        end = tempT[:3, 3]
        return MAP_X_MIN <= end[0] <= MAP_X_MAX and MAP_Y_MIN <= end[1] <= MAP_Y_MAX

### Tunnel Containment Check

Determines whether the robot is inside the tunnel, based on distances to the left/right walls and obstacles.


In [14]:
class TunnelBuilder(TunnelBuilder):
    def is_robot_inside_tunnel(self, robot_position, heading):
            robot_xy = np.array(robot_position[:2])
            right_dir = np.array([math.cos(heading - math.pi/2), math.sin(heading - math.pi/2)])
            left_dir = np.array([math.cos(heading + math.pi/2), math.sin(heading + math.pi/2)])
        
            def check_walls(walls, direction):
                for pos, _, _ in walls:
                    wall_xy = np.array(pos[:2])
                    to_wall = wall_xy - robot_xy
                    dist = np.linalg.norm(to_wall)
                    dot = np.dot(to_wall / (dist + 1e-6), direction)
                    if dot > 0.7:
                        expected = self.base_wall_distance
                        if abs(dist - expected) < 0.05:
                            return True
                return False
        
            def check_obstacles(obstacles):
                for pos, _, size in obstacles:
                    wall_xy = np.array(pos[:2])
                    dist = np.linalg.norm(wall_xy - robot_xy)
                    # Tolerância baseada na largura do obstáculo
                    if dist < size[1] / 2 + ROBOT_RADIUS + 0.02:
                        return True
                return False
        
            near_right = check_walls(self.right_walls, right_dir)
            near_left = check_walls(self.left_walls, left_dir)
            near_obstacle = check_obstacles(self.obstacles)
        
            inside = (near_right and near_left) or near_obstacle
            #print(f"[DEBUG] Inside tunnel: {inside} | Right: {near_right}, Left: {near_left}, Obstacle: {near_obstacle}")
            return inside


### SimulationManager – Initialization

This class manages the entire simulation workflow: building tunnels, placing the robot, running the experiment loop, processing sensor data, and collecting results. It is the high-level controller of the simulation process.


In [15]:
class SimulationManager:
    def __init__(self, supervisor):
        self.supervisor = supervisor
        self.timestep = int(self.supervisor.getBasicTimeStep())
        self.robot = self.supervisor.getFromDef(ROBOT_NAME)  # cuidado, ROBOT_NAME precisa existir
        if self.robot is None:
            raise ValueError(f"Robot with DEF name '{ROBOT_NAME}' not found.")
        self.translation = self.robot.getField("translation")
        self.lidar = self.supervisor.getDevice("lidar")
        self.lidar.enable(self.timestep)
        self.stats = {'total_collisions': 0, 'successful_runs': 0, 'failed_runs': 0}

### Running Experiments (Multiple Runs)

Runs the main experiment loop for a given number of runs. For each run, a random tunnel is generated, the robot is placed at the start, and lidar-based control is applied to navigate. Collisions and performance stats are recorded.


In [16]:
class SimulationManager(SimulationManager):
    def run_experiment(self, num_runs):
        # Run the experiment for the specified number of runs
        for run in range(num_runs):
            print(f"--- Starting Run {run + 1} ---")
    
            # Build a new random tunnel
            tunnel_builder = TunnelBuilder(self.supervisor)
            start_pos, end_pos, walls_added = tunnel_builder.build_tunnel(NUM_CURVES)
    
            # If tunnel building failed (out of bounds), skip this run
            if start_pos is None:
                print("Tunnel out of bounds, skipping run.")
                continue
    
            # Place the robot at the start of the tunnel and reset physics
            self.translation.setSFVec3f([start_pos[0], start_pos[1], ROBOT_RADIUS])
            self.robot.resetPhysics()
    
            # Get wheel motors and set them to velocity control mode (infinite position)
            left = self.supervisor.getDevice("left wheel motor")
            right = self.supervisor.getDevice("right wheel motor")
            left.setPosition(float('inf'))
            right.setPosition(float('inf'))
            left.setVelocity(0)  # Start with zero velocity
            right.setVelocity(0)
    
            collision_count = 0
            # Flag to avoid counting multiple collisions for the same event
            flag = False
            start_time = self.supervisor.getTime()  # Record simulation start time
    
            # Simulation loop
            while self.supervisor.step(self.timestep) != -1:
                # Timeout check
                if self.supervisor.getTime() - start_time > TIMEOUT_DURATION:
                    print("Timeout")
                    break
    
                # Get Lidar data
                data = self.lidar.getRangeImage()
    
                # Process Lidar data using the control logic
                lv, av = self._process_lidar(data)
    
                # Send velocity commands to the robot
                cmd_vel(self.supervisor, lv, av)
    
                # Get current robot position
                pos = np.array(self.translation.getSFVec3f())
    
                # Basic collision detection: deviation from centerline
                current_base_wall_distance = tunnel_builder.base_wall_distance
                current_distance_from_center = abs(pos[1])  # y-axis is considered as lateral deviation
    
                print(f"[DEBUG] Distance from center: {current_distance_from_center:.3f} | Allowed limit: {current_base_wall_distance - ROBOT_RADIUS:.3f}")
    
                if current_distance_from_center > current_base_wall_distance - ROBOT_RADIUS:
                    if not flag:
                        print(f"[WARNING] Robot left the tunnel! Position: {pos[:2]}")
                        flag = True
                else:
                    flag = False
    
                # Check if the robot reached the end of the tunnel
                if end_pos is not None and np.linalg.norm(pos[:2] - end_pos[:2]) < 0.1:
                    print(f"Reached end in {self.supervisor.getTime() - start_time:.1f}s")
                    break
    
            # Remove tunnel walls after the run
            self._remove_walls(walls_added)
            print ("slay")
    
            # Update statistics
            self.stats['total_collisions'] += collision_count
            if collision_count == 0 and end_pos is not None and np.linalg.norm(pos[:2] - end_pos[:2]) < 0.1:
                self.stats['successful_runs'] += 1
            else:
                self.stats['failed_runs'] += 1
    
            print(f"Run {run + 1} finished with {collision_count} collisions.")
    
        # Print final results
        self._print_summary()


### LiDAR-Based Wall-Following Control (for Experiments)

Processes raw LiDAR readings and computes linear and angular velocities using proportional control. This version is used in regular experiments (not GA).


In [17]:
class SimulationManager(SimulationManager):
    def _process_lidar_with_params(self, dist_values: [float], distP=10.0, angleP=7.0) -> (float, float):
    
            """
            Robot control logic based on Lidar data.
            Adapted from IRI - TP2 - Ex 4 by Gonçalo Leão.
            """
            # Assuming direction 1 for wall following (e.g., right wall)
            # You might need to adjust this or add logic to determine the direction
            # based on the tunnel structure or robot's initial position/goal.
            direction: int = 1
    
            maxSpeed: float = 0.1
            distP: float = 10.0  # Proportional gain for distance error
            angleP: float = 7.0  # Proportional gain for angle error
            wallDist: float = 0.1 # Desired distance to the wall
    
            # Find the angle of the ray that returned the minimum distance
            size: int = len(dist_values)
            if size == 0:
                # Handle case where lidar data is empty
                return 0.0, 0.0
    
            min_index: int = 0
            if direction == -1:
                min_index = size - 1
            for i in range(size):
                idx: int = i
                if direction == -1:
                    idx = size - 1 - i
                # Ensure distance is valid (greater than 0) before considering it minimum
                if dist_values[idx] < dist_values[min_index] and dist_values[idx] > 0.0:
                    min_index = idx
                # If the current min_index has an invalid distance, find the next valid one
                elif dist_values[min_index] <= 0.0 and dist_values[idx] > 0.0:
                     min_index = idx
    
    
            angle_increment: float = 2*math.pi / (size - 1) if size > 1 else 0.0
            angleMin: float = (size // 2 - min_index) * angle_increment
            distMin: float = dist_values[min_index]
    
            # Get distances from specific directions
            distFront: float = dist_values[size // 2] if size > 0 else float('inf')
            distSide: float = dist_values[size // 4] if size > 0 and size // 4 < size else float('inf') if (direction == 1) else dist_values[3*size // 4] if size > 0 and 3*size // 4 < size else float('inf')
            distBack: float = dist_values[0] if size > 0 else float('inf')
    
    
            # Prepare message for the robot's motors
            linear_vel: float
            angular_vel: float
    
            # print("distMin", distMin) # Commented out to reduce console output
            # print("angleMin", angleMin*180/math.pi) # Commented out
    
            # Decide the robot's behavior
            if math.isfinite(distMin):
                # Check for potential unblocking scenario (stuck)
                if distFront < 1.25*wallDist and (distSide < 1.25*wallDist or distBack < 1.25*wallDist):
                    # print("UNBLOCK") # Commented out
                    # Turn away from the detected obstacles
                    angular_vel = direction * -1 * maxSpeed # Use maxSpeed for turning velocity
                    linear_vel = 0 # Stop linear movement while unblocking
                else:
                    # print("REGULAR") # Commented out
                    # Calculate angular velocity based on distance and angle errors
                    # Error 1: Difference between minimum distance and desired wall distance
                    # Error 2: Difference between angle of minimum distance and desired angle (pi/2 for side wall)
                    angular_vel = direction * distP * (distMin - wallDist) + angleP * (angleMin - direction * math.pi / 2)
                    # print("angular_vel", angular_vel, " wall comp = ", direction * distP * (distMin - wallDist), ", angle comp = ", angleP * (angleMin - direction * math.pi / 2)) # Commented out
    
                # Adjust linear velocity based on front distance
                if distFront < wallDist:
                    # If obstacle is very close in front, stop and turn
                    # print("TURN") # Commented out
                    linear_vel = 0
                    # Angular velocity is already calculated above
                elif distFront < 2 * wallDist or distMin < wallDist * 0.75 or distMin > wallDist * 1.25:
                    # If obstacle is somewhat close or not at desired wall distance, slow down
                    # print("SLOW") # Commented out
                    linear_vel = 0.5 * maxSpeed
                    # Angular velocity is already calculated above
                else:
                    # Otherwise, cruise at max speed
                    # print("CRUISE") # Commented out
                    linear_vel = maxSpeed
                    # Angular velocity is already calculated above
            else:
                # If no finite minimum distance (e.g., open space), wander
                # print("WANDER") # Commented out
                # Use numpy.random.normal for random angular velocity
                angular_vel = np.random.normal(loc=0.0, scale=1.0) * maxSpeed # Scale random value by maxSpeed
                # print("angular_vel", angular_vel) # Commented out
                linear_vel = maxSpeed # Continue moving forward while wandering
    
            # Clamp angular velocity to a reasonable range if needed (optional but good practice)
            # angular_vel = max(-maxSpeed, min(maxSpeed, angular_vel))
    
    
            return linear_vel, angular_vel

### Removing Tunnel Walls After Each Run

Utility function to clean up the simulation world by removing the most recently added tunnel walls.


In [18]:
class SimulationManager(SimulationManager):
    def _remove_walls(self, count):
            # Remove the last 'count' children from the root node
            children = self.supervisor.getRoot().getField("children")
            for _ in range(count):
                children.removeMF(-1)

### Printing Final Results Summary

Displays the final summary after all simulation runs, including number of successes, failures, total collisions, and success rate.


In [19]:
class SimulationManager(SimulationManager):
    def _print_summary(self):
            # Print the final summary of the experiment
            print("\n=== Final Results ===")
            print(f"Successful runs: {self.stats['successful_runs']}")
            print(f"Failed runs: {self.stats['failed_runs']}")
            print(f"Total collisions: {self.stats['total_collisions']}")
            total_runs = self.stats['successful_runs'] + self.stats['failed_runs']
            if total_runs > 0:
                 print(f"Success rate: {self.stats['successful_runs'] / total_runs * 100:.1f}%")
            else:
                 print("No runs completed.")

### Lidar Processing for Genetic Algorithm Optimization

Similar to the previous lidar function, but this version allows fine-tuning of the distance and angle proportional gains (`distP` and `angleP`), making it suitable for optimization with a genetic algorithm.


In [20]:
class SimulationManager(SimulationManager):
    def _process_lidar(self, dist_values: [float], distP=10.0, angleP=7.0) -> (float, float):
    
            """
            Robot control logic based on Lidar data, with tunable distP and angleP parameters.
            Adapted for use with Genetic Algorithm optimization.
            """
            direction: int = 1  # Assuming right wall following
        
            maxSpeed: float = 0.1
            wallDist: float = 0.1  # Desired distance to the wall
        
            size: int = len(dist_values)
            if size == 0:
                return 0.0, 0.0  # No data available, stay still
        
            min_index: int = 0
            if direction == -1:
                min_index = size - 1
            for i in range(size):
                idx: int = i
                if direction == -1:
                    idx = size - 1 - i
                if dist_values[idx] < dist_values[min_index] and dist_values[idx] > 0.0:
                    min_index = idx
                elif dist_values[min_index] <= 0.0 and dist_values[idx] > 0.0:
                    min_index = idx
        
            angle_increment: float = 2 * math.pi / (size - 1) if size > 1 else 0.0
            angleMin: float = (size // 2 - min_index) * angle_increment
            distMin: float = dist_values[min_index]
        
            distFront: float = dist_values[size // 2] if size > 0 else float('inf')
            distSide: float = dist_values[size // 4] if (size > 0 and size // 4 < size) else float('inf') if (direction == 1) else dist_values[3 * size // 4] if (size > 0 and 3 * size // 4 < size) else float('inf')
            distBack: float = dist_values[0] if size > 0 else float('inf')
        
            linear_vel: float
            angular_vel: float
        
            if math.isfinite(distMin):
                if distFront < 1.25 * wallDist and (distSide < 1.25 * wallDist or distBack < 1.25 * wallDist):
                    angular_vel = direction * -1 * maxSpeed
                    linear_vel = 0
                else:
                    angular_vel = direction * distP * (distMin - wallDist) + angleP * (angleMin - direction * math.pi / 2)
        
                if distFront < wallDist:
                    linear_vel = 0
                elif distFront < 2 * wallDist or distMin < wallDist * 0.75 or distMin > wallDist * 1.25:
                    linear_vel = 0.5 * maxSpeed
                else:
                    linear_vel = maxSpeed
            else:
                angular_vel = np.random.normal(loc=0.0, scale=1.0) * maxSpeed
                linear_vel = maxSpeed
        
            
            return linear_vel, angular_vel

### Running Experiments with Tunable Parameters (GA Evaluation)

Runs three trials (easy, medium, hard) using specified `distP` and `angleP` values. Computes a fitness score based on how well the robot performs in each environment, considering tunnel completion, staying inside, and travel distance.


In [21]:
class SimulationManager(SimulationManager):
    # Behavior of a random parameter pair, not using predefined easy/medium/hard levels
    def run_experiment_with_params(self, distP: float, angleP: float) -> float:
        total_fitness = 0.0
        difficulties = ['easy', 'medium', 'hard']
    
        for difficulty in difficulties:
            print(f"\n--- Simulation with difficulty: {difficulty.upper()} ---")
    
            # Get fixed parameters for the selected difficulty
            num_curves, angle_range, clearance = get_difficulty_settings(difficulty)
    
            # Create the tunnel
            builder = TunnelBuilder(self.supervisor)
            start_pos, end_pos, walls_added = builder.build_tunnel(
                num_curves=num_curves,
                angle_range=angle_range,
                clearance=clearance
            )
    
            if start_pos is None:
                print(f"Tunnel out of bounds ({difficulty}). Partial fitness: -1000.")
                total_fitness += -1000.0
                continue
    
            # Reposition the robot
            self.translation.setSFVec3f([start_pos[0], start_pos[1], ROBOT_RADIUS])
            self.robot.resetPhysics()
    
            # Initialize motors
            left = self.supervisor.getDevice("left wheel motor")
            right = self.supervisor.getDevice("right wheel motor")
            left.setPosition(float('inf'))
            right.setPosition(float('inf'))
            left.setVelocity(0)
            right.setVelocity(0)
    
            # Initialize simulation variables
            grace_period = 2.0
            off_tunnel_events = 0
            flag_off_tunnel = False
            start_time = self.supervisor.getTime()
            timeout_occurred = False
            goal_reached = False
            distance_traveled_inside = 0.0
            previous_pos = np.array(self.translation.getSFVec3f())
            distance_traveled = 0.0
            last_pos = previous_pos.copy()
    
            while self.supervisor.step(self.timestep) != -1:
                if self.supervisor.getTime() - start_time > TIMEOUT_DURATION:
                    timeout_occurred = True
                    print("TIME OUT")
                    break
    
                data = self.lidar.getRangeImage()
                lv, av = self._process_lidar_with_params(data, distP, angleP)
                cmd_vel(self.supervisor, lv, av)
    
                pos = np.array(self.translation.getSFVec3f())
                distance_traveled += np.linalg.norm(pos[:2] - last_pos[:2])
                last_pos = pos.copy()
                rot = self.robot.getField("rotation").getSFRotation()
                heading = rot[3] if rot[2] >= 0 else -rot[3]
    
                inside_by_geometry = builder.is_robot_near_centerline(pos)
                inside_by_lidar = builder.is_robot_inside_tunnel(pos, heading)
                inside_tunnel = inside_by_geometry or inside_by_lidar
    
                if inside_tunnel:
                    distance_step = np.linalg.norm(pos[:2] - previous_pos[:2])
                    distance_traveled_inside += distance_step
                    flag_off_tunnel = False
                else:
                    if not flag_off_tunnel and self.supervisor.getTime() - start_time > grace_period:
                        off_tunnel_events += 1
                        flag_off_tunnel = True
                        print(f"[WARNING] Robot left the tunnel ({difficulty}): {pos[:2]}")
    
                previous_pos = pos
    
                if end_pos is not None and np.linalg.norm(pos[:2] - end_pos[:2]) < 0.1:
                    goal_reached = True
                    break
    
            # Remove tunnel walls before next difficulty
            self._remove_walls(walls_added)
            print ("slay")
    
            # Compute fitness for this difficulty
            fitness = 0.0
            total_tunnel_length = sum([seg[3] for seg in builder.segments])
            percent_traveled = distance_traveled / total_tunnel_length if total_tunnel_length > 0 else 0
    
            if percent_traveled >= 0.8:
                fitness += 500
            if goal_reached and not timeout_occurred:
                fitness += 1000
    
            fitness -= 2000 * off_tunnel_events
            fitness += 100 * distance_traveled_inside
    
            print(f"Fitness for {difficulty}: {fitness:.2f}")
            total_fitness += fitness
    
        # Average of the 3 difficulty levels
        average_fitness = total_fitness / len(difficulties)
        print(f"\nFinal average fitness: {average_fitness:.2f}")
        return average_fitness

### Running Robot on a Pre-Built Tunnel

Executes a simulation on a tunnel that has already been built externally. Useful for evaluating fitness or testing specific tunnel configurations multiple times without regeneration.


In [22]:
class SimulationManager(SimulationManager):
    def run_on_existing_tunnel(self, distP, angleP, builder, start_pos, end_pos):
            self.translation.setSFVec3f([start_pos[0], start_pos[1], ROBOT_RADIUS])
            self.robot.resetPhysics()
        
            left = self.supervisor.getDevice("left wheel motor")
            right = self.supervisor.getDevice("right wheel motor")
            left.setPosition(float('inf'))
            right.setPosition(float('inf'))
            left.setVelocity(0)
            right.setVelocity(0)
        
            grace_period = 2.0
            off_tunnel_events = 0
            flag_off_tunnel = False
            start_time = self.supervisor.getTime()
            timeout_occurred = False
            goal_reached = False
            distance_traveled_inside = 0.0
            previous_pos = np.array(self.translation.getSFVec3f())
            distance_traveled = 0.0
            last_pos = previous_pos.copy()
        
            while self.supervisor.step(self.timestep) != -1:
                if self.supervisor.getTime() - start_time > TIMEOUT_DURATION:
                    timeout_occurred = True
                    break
        
                data = self.lidar.getRangeImage()
                lv, av = self._process_lidar_with_params(data, distP, angleP)
                cmd_vel(self.supervisor, lv, av)
        
                pos = np.array(self.translation.getSFVec3f())
                distance_traveled += np.linalg.norm(pos[:2] - last_pos[:2])
                last_pos = pos.copy()
                rot = self.robot.getField("rotation").getSFRotation()
                heading = rot[3] if rot[2] >= 0 else -rot[3]
        
                inside_by_geometry = builder.is_robot_near_centerline(pos)
                inside_by_lidar = builder.is_robot_inside_tunnel(pos, heading)
                inside_tunnel = inside_by_geometry or inside_by_lidar
        
                if inside_tunnel:
                    distance_step = np.linalg.norm(pos[:2] - previous_pos[:2])
                    distance_traveled_inside += distance_step
                    flag_off_tunnel = False
                else:
                    if not flag_off_tunnel and self.supervisor.getTime() - start_time > grace_period:
                        off_tunnel_events += 1
                        flag_off_tunnel = True
        
                previous_pos = pos
        
                if end_pos is not None and np.linalg.norm(pos[:2] - end_pos[:2]) < 0.1:
                    goal_reached = True
                    break
        
            total_tunnel_length = sum([seg[3] for seg in builder.segments])
            percent_traveled = distance_traveled / total_tunnel_length if total_tunnel_length > 0 else 0
        
            fitness = 0.0
            if percent_traveled >= 0.8:
                fitness += 500
            if goal_reached and not timeout_occurred:
                fitness += 1000
            fitness -= 2000 * off_tunnel_events
            fitness += 100 * distance_traveled_inside
        
            return fitness
    

In [23]:
"""
if __name__ == "__main__":
    # Create a SimulationManager instance and run the experiment
    experiment = SimulationManager()
    experiment.run_experiment(10) # Reduced number of runs for quicker testing
"""


'\nif __name__ == "__main__":\n    # Create a SimulationManager instance and run the experiment\n    experiment = SimulationManager()\n    experiment.run_experiment(10) # Reduced number of runs for quicker testing\n'

### Individual Class – Genetic Representation

Defines an individual in the genetic algorithm population. Each individual has two genes: `distP` and `angleP`, which control the robot's wall-following behavior. The fitness is evaluated after simulation.


In [24]:
class Individual:
    def __init__(self, distP=None, angleP=None):
        # Genes (if not provided, initialize randomly)
        self.distP = distP if distP is not None else random.uniform(5.0, 15.0)
        self.angleP = angleP if angleP is not None else random.uniform(3.0, 10.0)
        self.fitness = None  # Fitness will be evaluated later
        
    def mutate(self, mutation_rate=0.1):
        """
        Small mutation in the genes (±10%).
        """
        if random.random() < mutation_rate:
            self.distP *= random.uniform(0.9, 1.1)
        if random.random() < mutation_rate:
            self.angleP *= random.uniform(0.9, 1.1)
    
    def crossover(self, other):
        """
        Simple crossover: averages the genes of both parents.
        """
        child_distP = (self.distP + other.distP) / 2.0
        child_angleP = (self.angleP + other.angleP) / 2.0
        return Individual(distP=child_distP, angleP=child_angleP)

    def get_genes(self):
        """
        Returns the genes (used to pass to the robot during simulation).
        """
        return self.distP, self.angleP

    def __repr__(self):
        """
        Returns a human-readable string of the individual for logging or debugging purposes.
        """
        return f"Individual(distP={self.distP:.2f}, angleP={self.angleP:.2f}, fitness={self.fitness})"

### Population Class – Genetic Algorithm Controller

This class manages a population of individuals for a genetic algorithm. It handles evaluation, selection, crossover, mutation, and evolution over generations.


In [25]:
class Population:
    def __init__(self, size, mutation_rate=0.1, elitism=1):
        self.size = size
        self.mutation_rate = mutation_rate
        self.elitism = elitism  # Number of top individuals to copy directly (elitism)
        self.individuals = [Individual() for _ in range(size)]
        
    def evaluate(self, simulator):
        """
        Evaluates the fitness all individuals in the population using the provided simulator.
        """
        for individual in self.individuals:
            distP, angleP = individual.get_genes()
            individual.fitness = simulator.run_with_parameters(distP, angleP)

    def select_parents(self):
        """
        Tournament selection: chooses the 2 individuals with the highest fitness.
        """
        tournament_size = 3
        parents = random.sample(self.individuals, tournament_size)
        parents.sort(key=lambda ind: ind.fitness, reverse=True)  # Higher fitness is better
        return parents[0], parents[1]

    def create_next_generation(self):
        """
        Creates the next generation using elitism + crossover + mutation.
        The top individuals are preserved, and the rest are generated through genetic operations.
        """
        # Sort the current population by fitness
        self.individuals.sort(key=lambda ind: ind.fitness, reverse=True)
    
        next_generation = []
    
        # Elitism: copy the best individuals
        for i in range(self.elitism):
            next_generation.append(self.individuals[i])
    
        # Generate the rest of the new generation
        while len(next_generation) < self.size:
            parent1, parent2 = self.select_parents()
            child = parent1.crossover(parent2)
            child.mutate(mutation_rate=self.mutation_rate)
            next_generation.append(child)
    
        self.individuals = next_generation

    def get_best_individual(self):
        """
        Returns the best individual in the population.
        """
        return max(self.individuals, key=lambda ind: ind.fitness)

### GeneticOptimizer – Optimization Loop

This class implements a simple Genetic Algorithm to optimize the control parameters `distP` and `angleP` for a robot. It evolves a population of parameter pairs over multiple generations based on simulation fitness.


In [26]:
class GeneticOptimizer:
    def __init__(self, simulation_manager, population_size=20, generations=10, mutation_rate=0.1):
        """
        Genetic Algorithm for optimizing robot navigation parameters.
        """
        self.simulation_manager = simulation_manager
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        
    def _create_individual(self):
            """
            Creates a random individual [distP, angleP].
            """
            distP = random.uniform(5.0, 20.0)  # Range for proportional gain on distance
            angleP = random.uniform(2.0, 10.0)  # Range for proportional gain on angle
            return [distP, angleP]

    def _mutate(self, individual):
            """
            Mutates an individual by adding small random noise.
            """
            if random.random() < self.mutation_rate:
                individual[0] += random.uniform(-2.0, 2.0)  # Mutate distP
            if random.random() < self.mutation_rate:
                individual[1] += random.uniform(-1.0, 1.0)  # Mutate angleP
            return individual

    def _crossover(self, parent1, parent2):
            """
            Creates two children from two parents using one-point crossover.
            """
            if random.random() < 0.5:
                child1 = [parent1[0], parent2[1]]
                child2 = [parent2[0], parent1[1]]
            else:
                child1 = [parent2[0], parent1[1]]
                child2 = [parent1[0], parent2[1]]
            return child1, child2

### Running the Genetic Algorithm

Executes the optimization process over a number of generations. In each generation, it evaluates all individuals, tracks the best one, and prints progress and fitness scores.


In [27]:
class GeneticOptimizer(GeneticOptimizer):
    def optimize(self):
        """
        Runs the Genetic Algorithm to optimize parameters.
        """
        best_individual = None
        best_fitness = -float("inf")
    
        for generation in range(self.generations):
            print(f"\n=== Generation {generation + 1} ===")
            
            population = [self._create_individual() for _ in range(self.population_size)]
            fitness_scores = []
    
            for idx, individual in enumerate(population):
                try:
                    distP, angleP = individual
                    print(f"\nIndividual {idx+1} → distP = {distP:.2f}, angleP = {angleP:.2f}")
                    average_fitness = self.simulation_manager.run_experiment_with_params(distP, angleP)
                    fitness_scores.append((average_fitness, individual))
    
                    if average_fitness > best_fitness:
                        best_fitness = average_fitness
                        best_individual = individual
                except Exception as e:
                    print(f"[ERROR] Failed to evaluate individual {idx+1}: {e}")
    
            print(f"[INFO] Generation {generation + 1} completed.")
    
        print(f"\nBest individual: distP = {best_individual[0]:.3f}, angleP = {best_individual[1]:.3f} with fitness = {best_fitness:.2f}")
        return best_individual


### Difficulty Settings Helper

Defines the difficulty settings for tunnel generation (number of curves, curve angle range, and tunnel clearance) based on a string input.


In [28]:
# Function to configure difficulty settings
def get_difficulty_settings(difficulty):
    if difficulty == 'easy':
        return 2, (math.radians(20), math.radians(30)), 4.0
    elif difficulty == 'hard':
        return 7, (math.radians(60), math.radians(100)), 2.0
    else:  # medium
        return 4, (math.radians(35), math.radians(60)), 3.0

### How does the Genetic Algorithm works?

The goal is to find the best values for the `distP`and `angleP`parameters which control the robot's movement. 
- `distP`: weight from the distance to the wall.
- `angleP`: orientation correction angle weight.

#### Structure of the Genetic Algorithm
1. **Individual**
   - Each individual in the population is a pair of parameters `(distP, angleP)` -> GENES.
2. **Fitness Function**
   - The quality of each pair is evaluated through the function: `run_experiment_with_params(distP, angleP) -> float`.
   - This function:
       - Creates tunnels with difficulties (`easy`, `medium`, `hard`).
       - Runs the simulation with the parameters.
       - Measures:
           -  If the robot reached the end (+1000 bonus).
           -  If the robot traveled more than 80% of the tunnel (+500 bonus).
           -  Penalties for leaving the tunnel (–2000 per event).
           -  Reward proportional to the distance traveled inside the tunnel (+100 per meter).
3. **Sensor processing (phenotype)**
   - The function `_process_lidar` translates the genetic parameters (`distP`,`angleP`) into linear and angular velocity commands: `linear_vel = ...\\angular_vel = distP * erro_dist + angleP * erro_ang`

4. **Selection, Crossover and Mutation**
   
    - **Selection**: tournament among 3 → selects the 2 best individuals.
    
    - **Crossover**: computes the average of the parents' genes.
    
    - **Mutation**: small perturbations/noise (±10%).
    
    - **Elitism**: preserves the top-performing individuals.

Basically:
```
[Initial Population]
         ↓ evaluation
    [Select best]
         ↓ crossover
    [Children generated]
         ↓ mutation
    [New population]
         ↓ repeats for N generations
```



### Neuroevolution with MLP Controller

In this approach, we replace manually tuned control parameters with a small feedforward neural network (MLP) that maps LiDAR input values directly to robot motion commands (linear and angular velocity). The network has one hidden layer and its weights and biases are encoded as a flat vector. A genetic algorithm is then used to evolve this vector, allowing the robot to learn optimal control behaviors through simulation-based fitness evaluation.


### MLPController class
This class represents a simple neural network with:
- 1 hidden layer (ReLU)
- Inputs: LIDAR values (e.g. 32 sensors)
- Outputs: 2 continuous values → linear and angular velocity
  
Main methods:
- set_weights(vector): loads the weights and bias from a linear vector.
- forward(x): performs inference with an input vector x.

In [29]:
class MLPController:
    def __init__(self, input_size, hidden_size, output_size, weights_vector=None):
        """
        Initializes a single-hidden-layer MLP with specified sizes.
        If weights_vector is given, it's used to set the weights and biases directly.
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Total number of weights and biases
        self.total_weights = (
            input_size * hidden_size +  # input-to-hidden weights
            hidden_size +               # hidden biases
            hidden_size * output_size + # hidden-to-output weights
            output_size                 # output biases
        )

        if weights_vector is not None:
            self.set_weights(weights_vector)
        else:
            # Initialize randomly if no weights are provided
            self.weights_input_hidden = np.random.randn(input_size, hidden_size)
            self.bias_hidden = np.random.randn(hidden_size)
            self.weights_hidden_output = np.random.randn(hidden_size, output_size)
            self.bias_output = np.random.randn(output_size)

    def set_weights(self, vector):
        """
        Sets the weights and biases from a flat vector.
        """
        assert len(vector) == self.total_weights, "Invalid weight vector size"
        idx = 0

        ih_size = self.input_size * self.hidden_size
        self.weights_input_hidden = vector[idx:idx+ih_size].reshape(self.input_size, self.hidden_size)
        idx += ih_size

        self.bias_hidden = vector[idx:idx+self.hidden_size]
        idx += self.hidden_size

        ho_size = self.hidden_size * self.output_size
        self.weights_hidden_output = vector[idx:idx+ho_size].reshape(self.hidden_size, self.output_size)
        idx += ho_size

        self.bias_output = vector[idx:idx+self.output_size]

    def forward(self, x):
        """
        Forward pass through the network.
        """
        hidden = np.dot(x, self.weights_input_hidden) + self.bias_hidden
        hidden = np.maximum(0, hidden)  # ReLU activation
        output = np.dot(hidden, self.weights_hidden_output) + self.bias_output
        return output  # Linear output (can apply tanh or clip later)


### IndividualNeural Class  
**Key Functions**:
- `get_genome()`: returns all weights and biases as a flat genome vector.  
- `mutate()`: applies Gaussian noise to some genes.  
- `crossover()`: performs arithmetic crossover between two individuals.  
- `act()`: uses the neural network to generate `[linear_vel, angular_vel]` from LiDAR input.  


In [30]:
class IndividualNeural:
    def __init__(self, input_size, hidden_size, output_size, weights_vector=None):
        """
        Represents an individual with a neural controller encoded as a flat weight vector.
        """
        self.controller = MLPController(input_size, hidden_size, output_size, weights_vector)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.fitness = None

    def get_genome(self):
        """
        Returns the genome (flattened weights and biases) of this individual.
        """
        return np.concatenate([
            self.controller.weights_input_hidden.flatten(),
            self.controller.bias_hidden,
            self.controller.weights_hidden_output.flatten(),
            self.controller.bias_output
        ])

    def mutate(self, mutation_rate=0.1, mutation_strength=0.1):
        """
        Applies Gaussian noise to some genes with a given probability.
        """
        genome = self.get_genome()
        for i in range(len(genome)):
            if random.random() < mutation_rate:
                genome[i] += np.random.normal(0, mutation_strength)
        self.controller.set_weights(genome)

    def crossover(self, other):
        """
        Performs arithmetic crossover between two individuals.
        """
        genome1 = self.get_genome()
        genome2 = other.get_genome()
        alpha = np.random.uniform(0, 1, size=genome1.shape)
        child_genome = alpha * genome1 + (1 - alpha) * genome2
        return IndividualNeural(self.input_size, self.hidden_size, self.output_size, child_genome)

    def act(self, lidar_input):
        """
        Executes the neural network on LIDAR input and returns the motor commands.
        """
        return self.controller.forward(lidar_input)

### NeuralPopulation Class  
This class manages the evolution of a population of neural network controllers using a genetic algorithm.

**Key Responsibilities**:
- **Initialization**: creates a population of `IndividualNeural` instances.  
- **evaluate()**: assigns a fitness score to each individual using a simulation environment.  
- **select_parents()**: uses tournament selection to choose two high-performing individuals.  
- **create_next_generation()**: generates a new population via elitism, crossover, and mutation.  
- **get_best_individual()**: retrieves the best individual from the current generation.
---------------------------------
**Aqui podemos reutilizar o codigo da classe Population e ajustar para a rede, mas para ja deixamos assim??**


In [31]:
class NeuralPopulation:
    def __init__(self, pop_size, input_size, hidden_size, output_size, mutation_rate=0.1, elitism=1):
        """
        Manages a population of neural individuals for neuroevolution.
        """
        self.pop_size = pop_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.mutation_rate = mutation_rate
        self.elitism = elitism

        self.individuals = [
            IndividualNeural(input_size, hidden_size, output_size)
            for _ in range(pop_size)
        ]

    def evaluate(self, simulator):
        """
        Evaluates the fitness of each individual using the provided simulator.
        """
        for individual in self.individuals:
            individual.fitness = simulator.evaluate(individual)

    def select_parents(self, tournament_size=3):
        """
        Selects two parents using tournament selection.
        """
        competitors = random.sample(self.individuals, tournament_size)
        competitors.sort(key=lambda ind: ind.fitness, reverse=True)
        return competitors[0], competitors[1]

    def create_next_generation(self):
        """
        Generates the next generation via elitism, crossover and mutation.
        """
        self.individuals.sort(key=lambda ind: ind.fitness, reverse=True)
        next_generation = self.individuals[:self.elitism]  # Elitism

        while len(next_generation) < self.pop_size:
            parent1, parent2 = self.select_parents()
            child = parent1.crossover(parent2)
            child.mutate(mutation_rate=self.mutation_rate)
            next_generation.append(child)

        self.individuals = next_generation

    def get_best_individual(self):
        """
        Returns the best individual in the current population.
        """
        return max(self.individuals, key=lambda ind: ind.fitness)


Same as the previous `run_experiment_with_params` but for the neural network.

In [32]:
class SimulationManager(SimulationManager):
    def run_experiment_with_network(self, individual) -> float:
        """
        Executes a simulation uquero sing a neural network-based individual (MLP controller).
        Computes the fitness based on classic evaluation metrics.
        """
        total_fitness = 0.0
        difficulties = ['easy', 'medium', 'hard']
    
        for difficulty in difficulties:
            print(f"\n--- Simulation with difficulty: {difficulty.upper()} ---")
    
            num_curves, angle_range, clearance = get_difficulty_settings(difficulty)
            builder = TunnelBuilder(self.supervisor)
            start_pos, end_pos, walls_added = builder.build_tunnel(num_curves, angle_range, clearance)
    
            if start_pos is None:
                print(f"Tunnel out of bounds ({difficulty}). Partial fitness: -1000.")
                total_fitness += -1000.0
                continue
    
            self.translation.setSFVec3f([start_pos[0], start_pos[1], ROBOT_RADIUS])
            self.robot.resetPhysics()
    
            left = self.supervisor.getDevice("left wheel motor")
            right = self.supervisor.getDevice("right wheel motor")
            left.setPosition(float('inf'))
            right.setPosition(float('inf'))
            left.setVelocity(0)
            right.setVelocity(0)
    
            grace_period = 2.0
            off_tunnel_events = 0
            flag_off_tunnel = False
            start_time = self.supervisor.getTime()
            timeout_occurred = False
            goal_reached = False
            distance_traveled_inside = 0.0
            previous_pos = np.array(self.translation.getSFVec3f())
            distance_traveled = 0.0
            last_pos = previous_pos.copy()
    
            while self.supervisor.step(self.timestep) != -1:
                if self.supervisor.getTime() - start_time > TIMEOUT_DURATION:
                    timeout_occurred = True
                    print("TIME OUT")
                    break
    
                lidar_data = self.lidar.getRangeImage()
                lidar_data = np.nan_to_num(lidar_data, nan=0.0)  # ensure clean input
                lv, av = individual.act(np.array(lidar_data))  # forward pass on MLP
                cmd_vel(self.supervisor, lv, av)
    
                pos = np.array(self.translation.getSFVec3f())
                distance_traveled += np.linalg.norm(pos[:2] - last_pos[:2])
                last_pos = pos.copy()
                rot = self.robot.getField("rotation").getSFRotation()
                heading = rot[3] if rot[2] >= 0 else -rot[3]
    
                inside_by_geometry = builder.is_robot_near_centerline(pos)
                inside_by_lidar = builder.is_robot_inside_tunnel(pos, heading)
                inside_tunnel = inside_by_geometry or inside_by_lidar
    
                if inside_tunnel:
                    distance_step = np.linalg.norm(pos[:2] - previous_pos[:2])
                    distance_traveled_inside += distance_step
                    flag_off_tunnel = False
                else:
                    if not flag_off_tunnel and self.supervisor.getTime() - start_time > grace_period:
                        off_tunnel_events += 1
                        flag_off_tunnel = True
                        print(f"[WARNING] Robot left the tunnel ({difficulty}): {pos[:2]}")
    
                previous_pos = pos
    
                if end_pos is not None and np.linalg.norm(pos[:2] - end_pos[:2]) < 0.1:
                    goal_reached = True
                    break
    
            self._remove_walls(walls_added)
    
            fitness = 0.0
            total_tunnel_length = sum([seg[3] for seg in builder.segments])
            percent_traveled = distance_traveled / total_tunnel_length if total_tunnel_length > 0 else 0
    
            if percent_traveled >= 0.8:
                fitness += 500
            if goal_reached and not timeout_occurred:
                fitness += 1000
            fitness -= 2000 * off_tunnel_events
            fitness += 100 * distance_traveled_inside
    
            print(f"Fitness for {difficulty}: {fitness:.2f}")
            total_fitness += fitness
    
        average_fitness = total_fitness / len(difficulties)
        print(f"\nFinal average fitness: {average_fitness:.2f}")
        return average_fitness


### Evolution Loop

This function performs the full neuroevolution process over a defined number of generations. It initializes a population of MLP-based controllers, evaluates their performance in simulation, applies genetic operations (selection, crossover, mutation, elitism), and returns the best evolved individual along with the fitness history across generations.

In [33]:
def run_neuroevolution(simulator, generations=30, pop_size=20,
                       input_size=32, hidden_size=16, output_size=2,
                       mutation_rate=0.1, elitism=1):
    """
    Main evolution loop for neuroevolution.
    Evolves a population of neural networks to control a robot using LIDAR input.
    """
    # Initialize population
    population = NeuralPopulation(pop_size, input_size, hidden_size, output_size,
                                   mutation_rate=mutation_rate, elitism=elitism)

    fitness_history = []

    for gen in range(generations):
        print(f"\nGeneration {gen + 1}/{generations}")

        # Evaluate current generation
        population.evaluate(simulator)

        # Log best fitness
        best = population.get_best_individual()
        fitness_history.append(best.fitness)
        print(f"Best fitness: {best.fitness:.2f}")

        # Reproduce next generation
        population.create_next_generation()

    print("\nEvolution completed!")
    return population.get_best_individual(), fitness_history


### Main Execution – Run Genetic Optimization

Initializes the Webots `Supervisor`, sets up the `SimulationManager`, and launches the `GeneticOptimizer`. It runs the optimization for a set number of generations and prints the best parameters found.


#### Main Novo para rede

In [34]:
"""if __name__ == "__main__":

    # Create Supervisor and SimulationManager instance
    supervisor = Supervisor()
    simulator = SimulationManager(supervisor)

    # Redirect evaluation to the MLP network
    simulator.evaluate = lambda individual: simulator.run_experiment_with_network(individual)

    # Get number of LIDAR sensors
    lidar_data = np.nan_to_num(simulator.lidar.getRangeImage(), nan=0.0)
    input_size = len(lidar_data)
    
    # Run evolution
    best_individual, fitness_history = run_neuroevolution(
        simulator=simulator,
        generations=30,
        pop_size=20,
        input_size=input_size,
        hidden_size=16,
        output_size=2,
        mutation_rate=0.1,
        elitism=1
)

    # Save logs
    np.savetxt("fitness_log.txt", fitness_history)
    np.save("best_genome.npy", best_individual.get_genome())

    # View evolution
    plot_fitness_evolution(fitness_history)

    # See the behavior of the best individual
    print("\nRunning final simulation with best individual...")
    simulator.run_experiment_with_network(best_individual)"""


'if __name__ == "__main__":\n\n    # Create Supervisor and SimulationManager instance\n    supervisor = Supervisor()\n    simulator = SimulationManager(supervisor)\n\n    # Redirect evaluation to the MLP network\n    simulator.evaluate = lambda individual: simulator.run_experiment_with_network(individual)\n\n    # Get number of LIDAR sensors\n    lidar_data = np.nan_to_num(simulator.lidar.getRangeImage(), nan=0.0)\n    input_size = len(lidar_data)\n\n    # Run evolution\n    best_individual, fitness_history = run_neuroevolution(\n        simulator=simulator,\n        generations=30,\n        pop_size=20,\n        input_size=input_size,\n        hidden_size=16,\n        output_size=2,\n        mutation_rate=0.1,\n        elitism=1\n)\n\n    # Save logs\n    np.savetxt("fitness_log.txt", fitness_history)\n    np.save("best_genome.npy", best_individual.get_genome())\n\n    # View evolution\n    plot_fitness_evolution(fitness_history)\n\n    # See the behavior of the best individual\n    p

#### Main antigo

In [ ]:
if __name__ == "__main__":
    supervisor = Supervisor()  # create a single Supervisor instance
    experiment = SimulationManager(supervisor)  # pass the supervisor to the simulation manager

    optimizer = GeneticOptimizer(
        experiment,
        population_size=10,
        generations=5,
        mutation_rate=0.2
    )

    best_params = optimizer.optimize()
    print(f"Best parameters found: distP = {best_params[0]:.3f}, angleP = {best_params[1]:.3f}")



=== Generation 1 ===

Individual 1 → distP = 6.93, angleP = 5.64

--- Simulation with difficulty: EASY ---
Building tunnel with clearance factor: 4.00
Skipping obstacle—too close to another.
[ERROR] Failed to evaluate individual 1: 'SimulationManager' object has no attribute '_process_lidar_with_params'

Individual 2 → distP = 10.07, angleP = 5.71

--- Simulation with difficulty: EASY ---
Building tunnel with clearance factor: 4.00


### Final Evaluation with Best Parameters

After optimization, we directly evaluate the best `distP` and `angleP` values found by the genetic algorithm to verify their performance in simulation.


In [ ]:
best_distP = 8.338
best_angleP = 5.328

fitness = experiment.run_experiment_with_params(best_distP, best_angleP)
print(f"Direct evaluation with best parameters → Fitness: {fitness:.2f}")
